In [0]:
%sql

USE CATALOG dlt_catalog;
USE SCHEMA dlt_schema;
CREATE or Replace VIEW event_log_raw AS SELECT * FROM event_log(TABLE(dlt_catalog.dlt_schema.raw_farmers_market));

In [0]:
df_events = spark.sql("""
          SELECT * FROM event_log(TABLE(dlt_catalog.dlt_schema.raw_farmers_market))
          """)

In [0]:
df_events.display()

id sequence origin timestamp message level maturity_level error details event_type 3c21d8c0-ade2-11ef-8457-00163e3daa30 List(List(execution, 1732837264208001), null) List(Azure, centralus, 362282074994262, null, 6fb47b49-ba9a-465d-8f7c-95f423557b9d, WORKSPACE, farmers market dlt pipeline, 1128-233914-fkcnmatm, 66dbe11f-96db-4bec-9366-125e631cb906, null, null, null, null, null, null, 66dbe11f-96db-4bec-9366-125e631cb906, null) 2024-11-28T23:41:05.484Z Update 66dbe1 is INITIALIZING. INFO STABLE null {"update_progress":{"state":"INITIALIZING"}} update_progress f6871730-ade1-11ef-8fb2-92c611654b16 List(List(execution, 1732837264208002), 1732837148708001) List(Azure, centralus, 362282074994262, null, 6fb47b49-ba9a-465d-8f7c-95f423557b9d, WORKSPACE, farmers market dlt pipeline, null, null, null, null, null, null, null, null, null, null) 2024-11-28T23:39:08.707Z User admin@mngenvmcap374485.onmicrosoft.com created pipeline. INFO STABLE null {"user_action":{"action":"CREATE","user_name":"admin@mngenvmcap374485.onmicrosoft.com","user_id":2830541782567654,"request":{"create_request":{"id":"6fb47b49-ba9a-465d-8f7c-95f423557b9d","pipeline_type":"WORKSPACE","name":"farmers market dlt pipeline","clusters":[{"label":"default","node_type_id":"Standard_DS3_v2","autoscale":{"min_workers":1,"max_workers":2,"mode":"ENHANCED"}}],"libraries":[{"notebook":{"path":"/Users/admin@mngenvmcap374485.onmicrosoft.com/dlt-helpers/integration-testing/dlt_integration_pipeline"}}],"target":"dlt_schema","continuous":false,"development":true,"photon":false,"edition":"ADVANCED","channel":"CURRENT","catalog":"dlt_catalog","data_sampling":false}}}} user_action f9eb29c0-ade1-11ef-8fb2-92c611654b16 List(List(execution, 1732837264208003), 1732837154396001) List(Azure, centralus, 362282074994262, null, 6fb47b49-ba9a-465d-8f7c-95f423557b9d, WORKSPACE, farmers market dlt pipeline, null, 66dbe11f-96db-4bec-9366-125e631cb906, null, null, null, null, null, null, 66dbe11f-96db-4bec-9366-125e631cb906, null) 2024-11-28T23:39:14.396Z User admin@mngenvmcap374485.onmicrosoft.com started an update. INFO STABLE null {"user_action":{"action":"START","user_name":"admin@mngenvmcap374485.onmicrosoft.com","user_id":2830541782567654,"request":{"start_request":{"full_refresh":false,"validate_only":false}}}} user_action f9f42a70-ade1-11ef-8fb2-92c611654b16 List(List(execution, 1732837264208004), 1732837154456001) List(Azure, centralus, 362282074994262, null, 6fb47b49-ba9a-465d-8f7c-95f423557b9d, WORKSPACE, farmers market dlt pipeline, null, 66dbe11f-96db-4bec-9366-125e631cb906, null, null, null, null, null, null, 66dbe11f-96db-4bec-9366-125e631cb906, null) 2024-11-28T23:39:14.455Z Update 66dbe1 started by API_CALL. INFO STABLE null {"create_update":{"cause":"API_CALL","config":{"id":"6fb47b49-ba9a-465d-8f7c-95f423557b9d","pipeline_type":"WORKSPACE","name":"farmers market dlt pipeline","configuration":{"pipelines.allowClearingTableComment":"true","pipelines.alterViewsInHMS.enabled":"true","pipelines.analysis.maybeResolveFlowsParallely":"true","pipelines.analysis.maybeResolvePureSQLPipelinesParallely":"true","pipelines.applyChanges.viewMetrics.enabled":"true","pipelines.applyChangesFromSnapshot.analyzeLambdaSourceWithoutPipelineContext":"true","pipelines.applyChangesFromSnapshot.castSnapshotToTargetSchema":"true","pipelines.applyChangesFromSnapshot.stateStoreFormat":"proto","pipelines.changeDataFeed.enabled":"true","pipelines.dataPlaneStructuredLogging.enabled":"true","pipelines.decomposition.enableOnSnapshotFlow":"true","pipelines.decomposition.enabled":"true","pipelines.dltDebugger.enableNodeStatusReporter":"true","pipelines.dltDebugger.nodeStatusEventLoggingEnabled":"false","pipelines.enableApplyChangesLineage.enabled":"true","pipelines.enableBlockListReconciliationQueryDefinition":"true","pipelines.enableContinuousPipelineTriggerValidation":"false","pipelines.enableCyclicDependencyDetection":"true","pipelines.enableEventLogTableIdWrites":"true","pipelines.enableLiveSchemaUsageValidation":"

In [0]:
%sql
--select * from event_log_raw;

SELECT id, `timestamp`, origin.flow_name, details:flow_progress.metrics.num_output_rows, details:flow_progress.status, details:flow_progress.data_quality.dropped_records, details:flow_progress.data_quality.expectations, * FROM event_log_raw WHERE event_type = 'flow_progress' ORDER BY timestamp DESC;

id,timestamp,flow_name,num_output_rows,status,dropped_records,expectations,id,sequence,origin,timestamp,message,level,maturity_level,error,details,event_type
5799dc10-ade2-11ef-8457-00163e3daa30,2024-11-28T23:41:51.569Z,pipelines.flowTimeMetrics.missingFlowName,null,null,null,null,5799dc10-ade2-11ef-8457-00163e3daa30,"List(List(execution, 1732837264208020), null)","List(Azure, centralus, 362282074994262, null, 6fb47b49-ba9a-465d-8f7c-95f423557b9d, WORKSPACE, farmers market dlt pipeline, 1128-233914-fkcnmatm, 66dbe11f-96db-4bec-9366-125e631cb906, null, null, null, null, pipelines.flowTimeMetrics.missingFlowName, null, 66dbe11f-96db-4bec-9366-125e631cb906, null)",2024-11-28T23:41:51.569Z,Reported flow time metrics for flowName: 'pipelines.flowTimeMetrics.missingFlowName'.,METRICS,STABLE,null,"{""flow_progress"":{""metrics"":{""executor_time_ms"":7109,""executor_cpu_time_ms"":5355}}}",flow_progress
57998df0-ade2-11ef-8457-00163e3daa30,2024-11-28T23:41:51.567Z,raw_farmers_market,null,null,null,null,57998df0-ade2-11ef-8457-00163e3daa30,"List(List(execution, 1732837264208019), null)","List(Azure, centralus, 362282074994262, null, 6fb47b49-ba9a-465d-8f7c-95f423557b9d, WORKSPACE, farmers market dlt pipeline, 1128-233914-fkcnmatm, 66dbe11f-96db-4bec-9366-125e631cb906, null, null, null, null, raw_farmers_market, null, 66dbe11f-96db-4bec-9366-125e631cb906, null)",2024-11-28T23:41:51.567Z,Reported flow time metrics for flowName: 'raw_farmers_market'.,METRICS,STABLE,null,"{""flow_progress"":{""metrics"":{""executor_time_ms"":4338,""executor_cpu_time_ms"":2667}}}",flow_progress
561dc360-ade2-11ef-8457-00163e3daa30,2024-11-28T23:41:49.078Z,raw_farmers_market,null,COMPLETED,null,null,561dc360-ade2-11ef-8457-00163e3daa30,"List(List(execution, 1732837264208017), null)","List(Azure, centralus, 362282074994262, null, 6fb47b49-ba9a-465d-8f7c-95f423557b9d, WORKSPACE, farmers market dlt pipeline, 1128-233914-fkcnmatm, 66dbe11f-96db-4bec-9366-125e631cb906, null, null, null, 7de39c70-282e-42de-b730-b2cfe37be8f1, raw_farmers_market, null, 66dbe11f-96db-4bec-9366-125e631cb906, null)",2024-11-28T23:41:49.078Z,Flow 'raw_farmers_market' has COMPLETED.,INFO,STABLE,null,"{""flow_progress"":{""status"":""COMPLETED""}}",flow_progress
56142670-ade2-11ef-8457-00163e3daa30,2024-11-28T23:41:49.015Z,raw_farmers_market,8518,RUNNING,0,"[{""name"":""state_not_null"",""dataset"":""raw_farmers_market"",""passed_records"":8517,""failed_records"":1},{""name"":""website_not_null"",""dataset"":""raw_farmers_market"",""passed_records"":5003,""failed_records"":3515},{""name"":""location_not_null"",""dataset"":""raw_farmers_market"",""passed_records"":3698,""failed_records"":4820}]",56142670-ade2-11ef-8457-00163e3daa30,"List(List(execution, 1732837264208016), null)","List(Azure, centralus, 362282074994262, null, 6fb47b49-ba9a-465d-8f7c-95f423557b9d, WORKSPACE, farmers market dlt pipeline, 1128-233914-fkcnmatm, 66dbe11f-96db-4bec-9366-125e631cb906, null, null, null, 7de39c70-282e-42de-b730-b2cfe37be8f1, raw_farmers_market, null, 66dbe11f-96db-4bec-9366-125e631cb906, null)",2024-11-28T23:41:49.015Z,Completed a streaming update of 'raw_farmers_market'.,INFO,STABLE,null,"{""flow_progress"":{""status"":""RUNNING"",""metrics"":{""num_output_rows"":8518,""backlog_bytes"":0,""backlog_files"":0},""data_quality"":{""dropped_records"":0,""expectations"":[{""name"":""state_not_null"",""dataset"":""raw_farmers_market"",""passed_records"":8517,""failed_records"":1},{""name"":""website_not_null"",""dataset"":""raw_farmers_market"",""passed_records"":5003,""failed_records"":3515},{""name"":""location_not_null"",""dataset"":""raw_farmers_market"",""passed_records"":3698,""failed_records"":4820}]}}}",flow_progress
4de82b90-ade2-11ef-8457-00163e3daa30,2024-11-28T23:41:35.305Z,raw_farmers_market,null,RUNNING,null,null,4de82b90-ade2-11ef-8457-00163e3daa30,"List(List(execution, 1732837264208015), null)","List(Azure, centralus, 362282074994262, null, 6fb47b49-ba9a-465d-8f7c-95f423557b9d, WORKSPACE, 

In [0]:
df_events.selectExpr(
    "id",
    "timestamp",
    "origin.flow_name",
    "details:flow_progress.metrics.num_output_rows",
    "details:flow_progress.status",
    "details:flow_progress.data_quality.dropped_records",
    "details:flow_progress.data_quality.expectations",
    "*",
).filter("event_type = 'flow_progress' and details:flow_progress.status = 'COMPLETED'").display()

id,timestamp,flow_name,num_output_rows,status,dropped_records,expectations,id,sequence,origin,timestamp,message,level,maturity_level,error,details,event_type
561dc360-ade2-11ef-8457-00163e3daa30,2024-11-28T23:41:49.078Z,raw_farmers_market,null,COMPLETED,null,null,561dc360-ade2-11ef-8457-00163e3daa30,"List(List(execution, 1732837264208017), null)","List(Azure, centralus, 362282074994262, null, 6fb47b49-ba9a-465d-8f7c-95f423557b9d, WORKSPACE, farmers market dlt pipeline, 1128-233914-fkcnmatm, 66dbe11f-96db-4bec-9366-125e631cb906, null, null, null, 7de39c70-282e-42de-b730-b2cfe37be8f1, raw_farmers_market, null, 66dbe11f-96db-4bec-9366-125e631cb906, null)",2024-11-28T23:41:49.078Z,Flow 'raw_farmers_market' has COMPLETED.,INFO,STABLE,null,"{""flow_progress"":{""status"":""COMPLETED""}}",flow_progress


In [0]:
df_expectations = df_events.selectExpr("id", "details:flow_progress.data_quality.expectations").filter("event_type = 'flow_progress' and details:flow_progress.status = 'RUNNING'")

df_expectations.display()

id,expectations
4de82b90-ade2-11ef-8457-00163e3daa30,null
56142670-ade2-11ef-8457-00163e3daa30,"[{""name"":""state_not_null"",""dataset"":""raw_farmers_market"",""passed_records"":8517,""failed_records"":1},{""name"":""website_not_null"",""dataset"":""raw_farmers_market"",""passed_records"":5003,""failed_records"":3515},{""name"":""location_not_null"",""dataset"":""raw_farmers_market"",""passed_records"":3698,""failed_records"":4820}]"


In [0]:
from pyspark.sql.types import (
    ArrayType,
    StructType,
    StructField,
    StringType,
    IntegerType,
)
from pyspark.sql.functions import explode, from_json

schema = ArrayType(
    StructType(
        [
            StructField("name", StringType()),
            StructField("dataset", StringType()),
            StructField("passed_records", IntegerType()),
            StructField("failed_records", IntegerType()),
        ]
    )
)

df_expectations = df_expectations.withColumn("expectations", explode(from_json(df_expectations.expectations, schema)))
df_expectations = df_expectations.selectExpr("id", "expectations.name",  "expectations.dataset", "expectations.passed_records", "expectations.failed_records")

df_expectations.display()

id,name,dataset,passed_records,failed_records
56142670-ade2-11ef-8457-00163e3daa30,state_not_null,raw_farmers_market,8517,1
56142670-ade2-11ef-8457-00163e3daa30,website_not_null,raw_farmers_market,5003,3515
56142670-ade2-11ef-8457-00163e3daa30,location_not_null,raw_farmers_market,3698,4820


### Explorations

In [0]:
spark.sql("""
CREATE OR REPLACE TEMP VIEW latest_update AS SELECT origin.update_id AS id FROM event_log_raw WHERE event_type = 'create_update' ORDER BY timestamp DESC LIMIT 1;
""").display()

In [0]:
spark.sql("""
          select * from latest_update;
          """).display()

id
66dbe11f-96db-4bec-9366-125e631cb906


In [0]:
df = spark.sql("""
SELECT
  row_expectations.dataset as dataset,
  row_expectations.name as expectation,
  SUM(row_expectations.passed_records) as passing_records,
  SUM(row_expectations.failed_records) as failing_records
FROM
  (
    SELECT
      explode(
        from_json(
          details :flow_progress.data_quality.expectations,
          "array<struct<name: string, dataset: string, passed_records: int, failed_records: int>>"
        )
      ) row_expectations
    FROM
      event_log_raw,
      latest_update
    WHERE
      event_type = 'flow_progress'
      AND origin.update_id = latest_update.id
  )
GROUP BY
  row_expectations.dataset,
  row_expectations.name
  """)

# # Write the DataFrame to a Unity Catalog table
# df.write.format("delta").saveAsTable("dlt_catalog.dlt_schema.quality_metrics")

display(df)

dataset,expectation,passing_records,failing_records
raw_farmers_market,location_not_null,3698,4820
raw_farmers_market,website_not_null,5003,3515
raw_farmers_market,state_not_null,8517,1
